In [1]:
df = spark.read.csv('/FileStore/tables/VeriFone_schema-fe6c7.csv',header=True,inferSchema=True)
df.cache()
df.printSchema()

root
-- VendorID: integer (nullable = true)
-- tpep_pickup_datetime: string (nullable = true)
-- tpep_dropoff_datetime: string (nullable = true)
-- passenger_count: integer (nullable = true)
-- trip_distance: double (nullable = true)
-- RatecodeID: integer (nullable = true)
-- store_and_fwd_flag: string (nullable = true)
-- PULocationID: integer (nullable = true)
-- DOLocationID: integer (nullable = true)
-- payment_type: integer (nullable = true)
-- fare_amount: double (nullable = true)
-- extra: double (nullable = true)
-- mta_tax: double (nullable = true)
-- tip_amount: double (nullable = true)
-- tolls_amount: double (nullable = true)
-- improvement_surcharge: double (nullable = true)
-- total_amount: double (nullable = true)
-- congestion_surcharge: double (nullable = true)

# Exercise 1

In [3]:
from pyspark.sql.functions import date_format
selecting_columns_df = df.select(date_format('tpep_dropoff_datetime',"HH").alias("hour"),
                                date_format('tpep_dropoff_datetime','E').alias("day"),'tip_amount')
selecting_columns_df.drop_duplicates()
selecting_columns_df.cache().printSchema()

root
-- hour: string (nullable = true)
-- day: string (nullable = true)
-- tip_amount: double (nullable = true)

In [4]:
by_hour = selecting_columns_df.groupBy("day","hour")
exercise1 = by_hour.avg("tip_amount").orderBy("day","hour")

In [5]:
exercise1_newColumns_df = exercise1.withColumnRenamed('avg(tip_amount)','avgTip')
exercise1_newColumns_df.printSchema()
exercise1_newColumns_df.cache().show(50)

root
-- day: string (nullable = true)
-- hour: string (nullable = true)
-- avgTip: double (nullable = true)

+---+----+------------------+
day|hour| avgTip|
+---+----+------------------+
Fri| 00|3.2483333333333313|
Fri| 01|3.0826136363636363|
Fri| 02| 3.033095238095238|
Fri| 03| 2.99695652173913|
Fri| 04| 4.024399999999999|
Fri| 05|2.7623529411764705|
Fri| 06| 2.179404761904763|
Fri| 07|2.2954787234042553|
Fri| 08|2.1877186311787074|
Fri| 09|2.4455805243445705|
Fri| 10| 2.540093023255813|
Fri| 11|2.4637765957446813|
Fri| 12| 2.664008810572687|
Fri| 13| 2.477285714285714|
Fri| 14|2.4409649122807004|
Fri| 15|2.4228755364806878|
Fri| 16| 2.728724279835391|
Fri| 17|2.7572556390977425|
Fri| 18| 2.431467889908256|
Fri| 19| 2.532422360248448|
Fri| 20| 2.617046263345196|
Fri| 21|2.7630241935483886|
Fri| 22|2.6962660944206007|
Fri| 23|2.8072457627118634|
Mon| 00| 2.907777777777777|
Mon| 01|3.5338888888888897|
Mon| 02|3.2804347826086957|
Mon| 03| 4.331538461538462|
Mon| 04| 3.536666666666667|
Mon| 05|2.9933333333333327|
Mon| 06|2.5469047619047616|
Mon| 07|2.0025414364640888|
Mon| 08|2.5027459016393445|
Mon| 09|2.4695454545454547|
Mon| 10| 2.497368421052631|
Mon| 11| 2.498465608465609|
Mon| 12| 2.365202020202021|
Mon| 13|2.7588780487804874|
Mon| 14|2.1526415094339626|
Mon| 15|2.5262931034482743|
Mon| 16| 2.506396396396395|
Mon| 17| 2.647142857142859|
Mon| 18|2.5580836236933813|
Mon| 19| 2.621896551724137|
Mon| 20| 2.663015873015874|
Mon| 21|2.7364186046511616|
Mon| 22|3.2133136094674555|
Mon| 23| 3.128267716535432|
Sat| 00|2.8073364485981296|
Sat| 01| 2.424198895027624|
+---+----+------------------+
only showing top 50 rows

# Exercise 2

In [7]:
exercise2_df = exercise1_newColumns_df
exercise2_df = exercise2_df.withColumnRenamed("avgTip","maxAvgTip")
exercise2_df.registerTempTable('table')
query = "SELECT hour, day, maxAvgTip FROM (SELECT *, MAX(maxAvgTip) OVER (PARTITION BY hour) AS MaxavgTip2 FROM table) M WHERE maxAvgTip=MaxavgTip2 ORDER BY hour"
exercise2_df_result = spark.sql(query).show(24)

+----+---+------------------+
hour|day| maxAvgTip|
+----+---+------------------+
 00|Thu|3.4306756756756744|
 01|Tue|3.6537499999999983|
 02|Mon|3.2804347826086957|
 03|Mon| 4.331538461538462|
 04|Thu| 4.253636363636364|
 05|Sat| 5.056111111111112|
 06|Sat| 3.909767441860466|
 07|Sun|3.3388461538461534|
 08|Sun| 2.599382716049382|
 09|Mon|2.4695454545454547|
 10|Tue|2.8058874458874454|
 11|Tue|2.6736186770428017|
 12|Wed| 2.797491166077738|
 13|Thu|3.2026938775510216|
 14|Thu| 2.745463917525774|
 15|Sun| 2.800684931506848|
 16|Wed| 3.096889763779529|
 17|Thu|2.8332110091743115|
 18|Sat| 3.199638989169676|
 19|Sun| 2.624130434782609|
 20|Mon| 2.663015873015874|
 21|Sun|3.1792982456140364|
 22|Mon|3.2133136094674555|
 23|Sun|3.5758139534883724|
+----+---+------------------+

#Both exercise together to verify it works before send it

In [9]:
# I made both exercises because and I upload both to you because I am using the first exercise result to make the Exercise 2

df = spark.read.csv('/FileStore/tables/VeriFone_schema-fe6c7.csv',header=True,inferSchema=True)
df.cache()
df.printSchema() # here I see that tip_amount is float/double

########### EXERCISE 1 ###########

from pyspark.sql.functions import date_format
selecting_columns_df = df.select(date_format('tpep_dropoff_datetime',"HH").alias("hour"),
                                date_format('tpep_dropoff_datetime','E').alias("day"),'tip_amount')
selecting_columns_df.drop_duplicates()
selecting_columns_df.cache().printSchema()
by_hour = selecting_columns_df.groupBy("day","hour")
exercise1 = by_hour.avg("tip_amount").orderBy("day","hour")
exercise1_newColumns_df = exercise1.withColumnRenamed('avg(tip_amount)','avgTip')
exercise1_newColumns_df.printSchema()
exercise1_newColumns_df.cache().show(50)

########### EXERCISE 2 ###########

exercise2_df = exercise1_newColumns_df
exercise2_df = exercise2_df.withColumnRenamed("avgTip","maxAvgTip")
exercise2_df.registerTempTable('table')
query = "SELECT hour, day, maxAvgTip FROM (SELECT *, MAX(maxAvgTip) OVER (PARTITION BY hour) AS MaxavgTip2 FROM table) M WHERE maxAvgTip=MaxavgTip2 ORDER BY hour"
exercise2_df_result = spark.sql(query).show(24)

root
-- VendorID: integer (nullable = true)
-- tpep_pickup_datetime: string (nullable = true)
-- tpep_dropoff_datetime: string (nullable = true)
-- passenger_count: integer (nullable = true)
-- trip_distance: double (nullable = true)
-- RatecodeID: integer (nullable = true)
-- store_and_fwd_flag: string (nullable = true)
-- PULocationID: integer (nullable = true)
-- DOLocationID: integer (nullable = true)
-- payment_type: integer (nullable = true)
-- fare_amount: double (nullable = true)
-- extra: double (nullable = true)
-- mta_tax: double (nullable = true)
-- tip_amount: double (nullable = true)
-- tolls_amount: double (nullable = true)
-- improvement_surcharge: double (nullable = true)
-- total_amount: double (nullable = true)
-- congestion_surcharge: double (nullable = true)

root
-- hour: string (nullable = true)
-- day: string (nullable = true)
-- tip_amount: double (nullable = true)

root
-- day: string (nullable = true)
-- hour: string (nullable = true)
-- avgTip: double (nullable = true)

+---+----+------------------+
day|hour| avgTip|
+---+----+------------------+
Fri| 00|3.2483333333333313|
Fri| 01|3.0826136363636363|
Fri| 02| 3.033095238095238|
Fri| 03| 2.99695652173913|
Fri| 04| 4.024399999999999|
Fri| 05|2.7623529411764705|
Fri| 06| 2.179404761904763|
Fri| 07|2.2954787234042553|
Fri| 08|2.1877186311787074|
Fri| 09|2.4455805243445705|
Fri| 10| 2.540093023255813|
Fri| 11|2.4637765957446813|
Fri| 12| 2.664008810572687|
Fri| 13| 2.477285714285714|
Fri| 14|2.4409649122807004|
Fri| 15|2.4228755364806878|
Fri| 16| 2.728724279835391|
Fri| 17|2.7572556390977425|
Fri| 18| 2.431467889908256|
Fri| 19| 2.532422360248448|
Fri| 20| 2.617046263345196|
Fri| 21|2.7630241935483886|
Fri| 22|2.6962660944206007|
Fri| 23|2.8072457627118634|
Mon| 00| 2.907777777777777|
Mon| 01|3.5338888888888897|
Mon| 02|3.2804347826086957|
Mon| 03| 4.331538461538462|
Mon| 04| 3.536666666666667|
Mon| 05|2.9933333333333327|
Mon| 06|2.5469047619047616|
Mon| 07|2.0025414364640888|
Mon| 08|2.5027459016393445|
Mon| 09|2.4695454545454547|
Mon| 10| 2.497368421052631|
Mon| 11| 2.498465608465609|
Mon| 12| 2.365202020202021|
Mon| 13|2.7588780487804874|
Mon| 14|2.1526415094339626|
Mon| 15|2.5262931034482743|
Mon| 16| 2.506396396396395|
Mon| 17| 2.647142857142859|
Mon| 18|2.5580836236933813|
Mon| 19| 2.621896551724137|
Mon| 20| 2.663015873015874|
Mon| 21|2.7364186046511616|
Mon| 22|3.2133136094674555|
Mon| 23| 3.128267716535432|
Sat| 00|2.8073364485981296|
Sat| 01| 2.424198895027624|
+---+----+------------------+
only showing top 50 rows

+----+---+------------------+
hour|day| maxAvgTip|
+----+---+------------------+
 00|Thu|3.4306756756756744|
 01|Tue|3.6537499999999983|
 02|Mon|3.2804347826086957|
 03|Mon| 4.331538461538462|
 04|Thu| 4.253636363636364|
 05|Sat| 5.056111111111112|
 06|Sat| 3.909767441860466|
 07|Sun|3.3388461538461534|
 08|Sun| 2.599382716049382|
 09|Mon|2.4695454545454547|
 10|Tue|2.8058874458874454|
 11|Tue|2.6736186770428017|
 12|Wed| 2.797491166077738|
 13|Thu|3.2026938775510216|
 14|Thu| 2.745463917525774|
 15|Sun| 2.800684931506848|
 16|Wed| 3.096889763779529|
 17|Thu|2.8332110091743115|
 18|Sat| 3.199638989169676|
 19|Sun| 2.624130434782609|
 20|Mon| 2.663015873015874|
 21|Sun|3.1792982456140364|
 22|Mon|3.2133136094674555|
 23|Sun|3.5758139534883724|
+----+---+------------------+